# Phonon computations

This is a quick sketch how to run a simple phonon calculation using DFTK.

> **Preliminary implementation**
>
> Practical phonon computations have only seen rudimentary testing as of now.
> As of now we do not yet recommend relying on this feature for production
> calculations. We appreciate any issues, bug reports or PRs.

First we run an SCF calculation.

In [1]:
using AtomsBuilder
using DFTK
using PseudoPotentialData

pseudopotentials = PseudoFamily("cp2k.nc.sr.lda.v0_1.semicore.gth")
model  = model_DFT(bulk(:Si); pseudopotentials, functionals=LDA())
basis  = PlaneWaveBasis(model; Ecut=10, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);
nothing  # hide

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.916436014337                   -0.69    4.9    120ms
  2   -7.921207690573       -2.32       -1.52    1.0   58.6ms
  3   -7.921405081381       -3.70       -2.49    1.4    110ms
  4   -7.921440949222       -4.45       -2.83    2.4   80.5ms
  5   -7.921441638484       -6.16       -3.02    1.0   59.5ms
  6   -7.921442006209       -6.43       -4.53    1.0   60.4ms
  7   -7.921442021908       -7.80       -4.69    3.0   93.4ms
  8   -7.921442022132       -9.65       -5.36    1.0   61.1ms
  9   -7.921442022135      -11.56       -5.14    1.8   95.3ms
 10   -7.921442022144      -11.04       -5.96    1.0   59.8ms
 11   -7.921442022144      -12.38       -7.00    1.2    238ms
 12   -7.921442022144      -13.43       -7.01    2.6   83.0ms
 13   -7.921442022144      -15.05       -7.50    1.0   61.2ms
 14   -7.921442022144   +    -Inf       -7.70    1.0   63.0ms
 15   -7.

Next we compute the phonon modes at the q-point `[1/4, 1/4, 1/4]`.

In [2]:
scfres = DFTK.unfold_bz(scfres)
phret_q0 = @time DFTK.phonon_modes(scfres; q=[0.25, 0.25, 0.25]);
nothing  # hide

Iter  Restart  Krydim  log10(res)  avg(CG)  Δtime   Comment
----  -------  ------  ----------  -------  ------  ---------------
                                      73.0   2.05s  Non-interacting
   1        0       1        0.31     52.5   9.52s  
   2        0       2       -0.52     47.0   1.46s  
   3        0       3       -2.40     41.8   1.28s  
   4        0       4       -4.46     29.7   1.02s  
   5        0       5       -6.46     17.4   743ms  
   6        0       6       -8.91      4.0   438ms  
   7        1       1       -7.83     55.7   2.04s  Restart
   8        1       2       -8.95      5.7   480ms  
                                      72.8   1.81s  Final orbitals
Iter  Restart  Krydim  log10(res)  avg(CG)  Δtime   Comment
----  -------  ------  ----------  -------  ------  ---------------
                                      73.0   1.62s  Non-interacting
   1        0       1        0.31     52.5   1.56s  
   2        0       2       -0.52     47.0   1.35s  
   3

These are the final phonon frequencies:

In [3]:
phret_q0.frequencies

6-element Vector{Float64}:
 -0.0034981553553114996
 -0.0029774511539486506
 -0.0029774511536649942
  0.004302230625188227
  0.004302230625539438
  0.004353201757231817